# 1. Configuração do Ambiente e Parâmetros Globais
Instalação de dependências, importação de bibliotecas e definição de constantes globais.
Estabelece o ano de referência, diretórios de saída e mapeia os códigos das tabelas do Anuário USP para facilitar a manutenção e reconfiguração anual do sistema.

In [16]:
# Instalação silenciosa de dependências
try:
    import openpyxl
except ImportError:
    %pip install openpyxl

import pandas as pd
import requests
import os

# --- Parâmetros Gerais ---
ANO_REFERENCIA = 2024
DIRETORIO_SAIDA = f'dados_{ANO_REFERENCIA}'

# --- Definição de Unidades (Campus São Carlos) ---
UNIDADES_SAO_CARLOS = ['EESC', 'IAU', 'ICMC', 'IFSC', 'IQSC']
UNIDADES_POS_GRAD = UNIDADES_SAO_CARLOS + ['ICMC e UFSCar', 'Interunidades em Bioengenharia']
UNIDADES_EXTENSAO = UNIDADES_SAO_CARLOS + ['PUSP-SC', 'Interunidades em Bioengenharia']

# --- Mapeamento de Tabelas (Fonte: USP Digital) ---
CONFIG_TABELAS = {
    "area_territorial": "T8.01",
    "servidores": "T1.02",          # Base para Servidores, Docentes e Alunos
    "professores_seniores": "T2.26",
    "pos_graduacao_evolucao": "T3.12",
    "extensao_especializacao": "T2.23",
    "extensao_presencial": "T3.15",
    "extensao_distancia": "T3.35"
}

# Garante a existência do diretório de saída
if not os.path.exists(DIRETORIO_SAIDA):
    os.makedirs(DIRETORIO_SAIDA)
    print(f"📁 Diretório de saída configurado: '{DIRETORIO_SAIDA}'")

# Compatibilidade com scripts legados
ano = ANO_REFERENCIA

# 2. Função Core de ETL (Extração)
Módulo reutilizável responsável pela conexão com o portal USP Digital.
Realiza o download dos arquivos `.xls`, valida a resposta do servidor e carrega os dados brutos em DataFrames do Pandas, utilizando a engine `xlrd` para compatibilidade com formatos legados.

In [17]:
def carregar_tabela_anuario_usp(ano, numero_tabela, verbose=True):
    """
    Baixa e carrega uma tabela do Anuário USP com tratamento de exceções.
    """
    url_excel = f'https://uspdigital.usp.br/anuario/br/tabelas/XLS/{ano}/{numero_tabela}.xls'
    nome_arquivo_local = f'tabela_{numero_tabela}_{ano}.xls'

    try:
        resposta = requests.get(url_excel, timeout=30)
        resposta.raise_for_status()

        with open(nome_arquivo_local, 'wb') as f:
            f.write(resposta.content)

        if verbose:
            print(f'⬇️ Download concluído: {nome_arquivo_local}')

        return pd.read_excel(nome_arquivo_local, skiprows=5, header=1, engine='xlrd')

    except Exception as e:
        print(f"❌ Erro de Extração ({numero_tabela}/{ano}): {e}")
        return None

## **ÁREA TERRITORIAL**

# 3. Indicadores de Infraestrutura: Área Territorial
Extração e processamento dos dados de área construída e terreno. Filtra especificamente as áreas do Campus de São Carlos (Área 1, Área 2), CDCC e CRHEA, calculando os totais consolidados.

In [18]:
print("--- Processando: Área Territorial ---")
df_anuario = carregar_tabela_anuario_usp(ANO_REFERENCIA, CONFIG_TABELAS['area_territorial'])

if df_anuario is not None:
    # Filtragem geográfica
    filtro_campus = df_anuario['Local'].isin(['Campus de São Carlos', 'Campus de São Carlos II', 'Centro de Divulgação Científica e Cultural (CDCC)', 'Itirapina - Centro de Recursos Hídricos e Ecologia Aplicada'])
    df_sao_carlos = df_anuario[filtro_campus]

    # Construção da tabela formatada
    dados = [
        ['Área edificada e territorial', '', ''],
        ['São Carlos', 'Área construída (em m²)', 'Área de terreno (em m²)'],
        ['Área 1', f"{df_sao_carlos[df_sao_carlos['Local'] == 'Campus de São Carlos']['Total (em m2)'].iloc[0]:.2f}", f"{df_sao_carlos[df_sao_carlos['Local'] == 'Campus de São Carlos']['Total (em m2)'].iloc[1]:.2f}"],
        ['Área 2', f"{df_sao_carlos[df_sao_carlos['Local'] == 'Campus de São Carlos II']['Total (em m2)'].iloc[0]:.2f}", f"{df_sao_carlos[df_sao_carlos['Local'] == 'Campus de São Carlos II']['Total (em m2)'].iloc[1]:.2f}"],
        ['CDCC', f"{df_sao_carlos[df_sao_carlos['Local'] == 'Centro de Divulgação Científica e Cultural (CDCC)']['Total (em m2)'].iloc[0]:.2f}", f"{df_sao_carlos[df_sao_carlos['Local'] == 'Centro de Divulgação Científica e Cultural (CDCC)']['Total (em m2)'].iloc[1]:.2f}"],
        ['CRHEA', f"{df_sao_carlos[df_sao_carlos['Local'] == 'Itirapina - Centro de Recursos Hídricos e Ecologia Aplicada']['Total (em m2)'].iloc[0]:.2f}", f"{df_sao_carlos[df_sao_carlos['Local'] == 'Itirapina - Centro de Recursos Hídricos e Ecologia Aplicada']['Total (em m2)'].iloc[1]:.2f}"],
        ['Total',
         f"{df_sao_carlos[df_sao_carlos['Tipo de Área'] == 'Área construída']['Total (em m2)'].sum():.2f}",
         f"{df_sao_carlos[df_sao_carlos['Tipo de Área'] == 'Área de terreno']['Total (em m2)'].sum():.2f}"]
    ]

    df_area = pd.DataFrame(dados)
    display(df_area)

--- Processando: Área Territorial ---
⬇️ Download concluído: tabela_T8.01_2024.xls


,0,1,2
0,Área edificada e territorial,,
1,São Carlos,Área construída (em m²),Área de terreno (em m²)
2,Área 1,166397.23,321457.00
3,Área 2,46668.80,978027.85
4,CDCC,0.00,3362.71
5,CRHEA,4397.35,253602.72
6,Total,217463.38,1556450.28


## **SERVIDORES**

# 4. Recursos Humanos: Servidores Técnicos-Administrativos
Contabilização do quadro de servidores não-docentes ativos, segmentado por unidade (EESC, IAU, ICMC, IFSC, IQSC) e órgãos suplementares (CDCC, PUSP-SC).

In [19]:
print("--- Processando: Servidores ---")
df_servidores_raw = carregar_tabela_anuario_usp(ANO_REFERENCIA, CONFIG_TABELAS['servidores'])

if df_servidores_raw is not None:
    # Filtros rigorosos conforme original
    df_filtrado = df_servidores_raw[
        (df_servidores_raw['EACH'].isin(UNIDADES_EXTENSAO + ['CDCC'])) &
        (df_servidores_raw['Docentes'] == 'Não-docentes') &
        (df_servidores_raw['Recursos'] == 'Recursos')
    ].copy()

    col_valor = df_filtrado.columns[-1]
    dados_csv = [['Servidores', ''], ['Unidades', 'Número']]
    total = 0

    ordem_apresentacao = ['EESC', 'IAU', 'ICMC', 'IFSC', 'IQSC', 'CDCC', 'PUSP-SC']

    for unidade in ordem_apresentacao:
        registro = df_filtrado[df_filtrado['EACH'] == unidade]
        num = int(registro[col_valor].iloc[0]) if not registro.empty else 0
        dados_csv.append([unidade, num])
        total += num

    dados_csv.append(['Total Unidades', total])
    df_final_servidores_csv = pd.DataFrame(dados_csv)
    display(df_final_servidores_csv)

--- Processando: Servidores ---
⬇️ Download concluído: tabela_T1.02_2024.xls


,0,1
0,Servidores,
1,Unidades,Número
2,EESC,290
3,IAU,37
4,ICMC,99
5,IFSC,156
6,IQSC,114
7,CDCC,28
8,PUSP-SC,201
9,Total Unidades,925


## **DOCENTES**

# 5. Recursos Humanos: Corpo Docente
Levantamento do número de docentes ativos nas unidades de ensino e pesquisa do campus.

In [20]:
print("--- Processando: Docentes ---")
# Reutiliza tabela carregada para otimização
if 'df_servidores_raw' not in locals():
    df_servidores_raw = carregar_tabela_anuario_usp(ANO_REFERENCIA, CONFIG_TABELAS['servidores'])

if df_servidores_raw is not None:
    df_filtrado = df_servidores_raw[
        (df_servidores_raw['EACH'].isin(UNIDADES_SAO_CARLOS)) &
        (df_servidores_raw['Docentes'] == 'Docentes') &
        (df_servidores_raw['Recursos'] == 'Recursos')
    ].copy()

    col_valor = df_filtrado.columns[-1]
    dados_csv = [['Docentes', ''], ['Unidades', 'Número']]
    total = 0

    for unidade in UNIDADES_SAO_CARLOS:
        registro = df_filtrado[df_filtrado['EACH'] == unidade]
        num = int(registro[col_valor].iloc[0]) if not registro.empty else 0
        dados_csv.append([unidade, num])
        total += num

    dados_csv.append(['Total', total])
    df_final_docentes_csv = pd.DataFrame(dados_csv)
    display(df_final_docentes_csv)

--- Processando: Docentes ---


,0,1
0,Docentes,
1,Unidades,Número
2,EESC,173
3,IAU,33
4,ICMC,124
5,IFSC,77
6,IQSC,56
7,Total,463


## **PROFESSORES SENIORES**

# 6. Recursos Humanos: Professores Seniores
Extração de dados da tabela específica (T2.26) referente aos Professores Seniores em atividade nas unidades.

In [21]:
print("--- Processando: Professores Seniores ---")
df_seniores = carregar_tabela_anuario_usp(ANO_REFERENCIA, CONFIG_TABELAS['professores_seniores'])

if df_seniores is not None:
    df_filtrado = df_seniores[
        (df_seniores['tipatcpfe'] == 'Professor Sênior') &
        (df_seniores['sglanr'].isin(UNIDADES_SAO_CARLOS))
    ].copy()

    col_total = 'total'
    dados_csv = [['Professores Seniores', ''], ['Unidades', 'Número']]
    total = 0

    for unidade in UNIDADES_SAO_CARLOS:
        registro = df_filtrado[df_filtrado['sglanr'] == unidade]
        num = int(registro[col_total].iloc[0]) if not registro.empty else 0
        dados_csv.append([unidade, num])
        total += num

    dados_csv.append(['Total', total])
    df_final_professores_seniores_csv = pd.DataFrame(dados_csv)
    display(df_final_professores_seniores_csv)

--- Processando: Professores Seniores ---
⬇️ Download concluído: tabela_T2.26_2024.xls


,0,1
0,Professores Seniores,
1,Unidades,Número
2,EESC,22
3,IAU,2
4,ICMC,12
5,IFSC,19
6,IQSC,1
7,Total,56


## **GRADUAÇÃO**

# 7. Graduação: Alunos Matriculados (Ano Base)
Consolidação do número de alunos de graduação matriculados no ano de referência. Inclui lógica específica para contabilizar programas conjuntos (Engenharia de Computação e Licenciatura em Ciências Exatas).

In [22]:
print("--- Processando: Graduação (Snapshot Atual) ---")
# A tabela T1.02 contém dados gerais de humanos
df_grad = carregar_tabela_anuario_usp(ANO_REFERENCIA, CONFIG_TABELAS['servidores'])

if df_grad is not None:
    col_num = df_grad.columns[-1]

    # Filtro Base para Unidades
    df_filt = df_grad[
        (df_grad['Humanos'] == 'Alunos cadastrados') &
        (df_grad['Docentes'] == 'Graduação')
    ].copy()

    dados_csv = [['Alunos de Graduação', ''], ['Unidades', 'Número']]
    total_geral = 0

    # 1. Processamento Unidades Individuais
    for unidade in UNIDADES_SAO_CARLOS:
        val = df_filt[df_filt['EACH'] == unidade][col_num]
        num = int(val.iloc[0]) if not val.empty else 0
        dados_csv.append([unidade, num])
        total_geral += num

    # 2. Processamento Programas Conjuntos
    dados_csv.append(['Programas Conjuntos', ''])

    def get_conjunto(nome_each):
        val = df_filt[
            (df_filt['A - Ensino e Pesquisa'] == 'G - Programas Conjuntos') &
            (df_filt['EACH'] == nome_each)
        ][col_num]
        return int(val.iloc[0]) if not val.empty else 0

    n_eng = get_conjunto('EESC e ICMC')
    dados_csv.append(['EESC e ICMC - Eng. Computação', n_eng])

    n_lic = get_conjunto('Interunidades Licenciatura São Carlos')
    dados_csv.append(['Licenciatura Ciência Exatas', n_lic])

    total_geral += n_eng + n_lic
    dados_csv.append(['Total', total_geral])

    df_final_graduacao_csv = pd.DataFrame(dados_csv)
    display(df_final_graduacao_csv)

--- Processando: Graduação (Snapshot Atual) ---
⬇️ Download concluído: tabela_T1.02_2024.xls


,0,1
0,Alunos de Graduação,
1,Unidades,Número
2,EESC,2596
3,IAU,285
4,ICMC,1306
5,IFSC,470
6,IQSC,306
7,Programas Conjuntos,
8,EESC e ICMC - Eng. Computação,310
9,Licenciatura Ciência Exatas,136


## **GRADUAÇÃO - EVOLUÇÃO**

# 8. Graduação: Evolução Histórica (Série Temporal)
Pipeline de extração múltipla para consolidar a série histórica de alunos de graduação (2019 até o ano atual).
O script executa iterações de extração, normaliza nomes de colunas dinamicamente, pivota os dados (Unidade x Ano) e formata a saída para compatibilidade com relatórios institucionais.

In [23]:
print("--- Processando: Graduação (Evolução Histórica) ---")

# Configuração da Série
cod_tabela_hist = CONFIG_TABELAS['servidores']
ano_inicial = 2019
anos_para_carregar = list(range(ano_inicial, ANO_REFERENCIA + 1))
lista_dfs_evolucao = []

print(f"⏳ Iniciando coleta de série histórica ({ano_inicial}-{ANO_REFERENCIA})...")

# --- 1. Extração (Loop) ---
for ano_loop in anos_para_carregar:
    df_temp = carregar_tabela_anuario_usp(ano_loop, cod_tabela_hist, verbose=False)

    if df_temp is not None:
        # Estratégia para encontrar coluna de total (última numérica)
        colunas_numericas = df_temp.select_dtypes(include=['number']).columns
        if len(colunas_numericas) > 0:
            col_total = colunas_numericas[-1]
            df_temp.rename(columns={col_total: 'Total_Alunos'}, inplace=True)

            # Adiciona coluna temporal APÓS renomear o total para evitar conflito
            df_temp['Ano'] = ano_loop
            lista_dfs_evolucao.append(df_temp)

# --- 2. Transformação e Pivotagem ---
if lista_dfs_evolucao:
    df_combinado = pd.concat(lista_dfs_evolucao, ignore_index=True)

    # Lista de interesse para a série
    ordem_itens = UNIDADES_SAO_CARLOS + ['EESC e ICMC', 'Interunidades Licenciatura São Carlos']

    # Filtro
    df_filtrado = df_combinado[
        (df_combinado['Humanos'] == 'Alunos cadastrados') &
        (df_combinado['Docentes'] == 'Graduação') &
        (df_combinado['EACH'].isin(ordem_itens))
    ].copy()

    # Pivot Table (Linhas: Unidades, Colunas: Anos)
    df_pivot = df_filtrado.pivot_table(
        index='EACH', columns='Ano', values='Total_Alunos', aggfunc='sum'
    ).reindex(ordem_itens).fillna(0).astype(int)

    # Formatação Final
    nomes_exibicao = {
        'EESC e ICMC': 'EESC e ICMC - Eng. Computação',
        'Interunidades Licenciatura São Carlos': 'Licenciatura Ciências Exatas'
    }

    dados_formatados = [
        ['Evolução do número de Alunos Matriculados de Graduação'] + [''] * len(anos_para_carregar),
        ['Unidades', 'Número - Anos'] + [''] * (len(anos_para_carregar) - 1),
        ['', ''] + [str(a) for a in anos_para_carregar]
    ]

    for unidade in ordem_itens:
        if unidade == 'EESC e ICMC':
            dados_formatados.append(['Programas Conjuntos', ''] + [''] * len(anos_para_carregar))

        nome_final = nomes_exibicao.get(unidade, unidade)
        valores_ano = df_pivot.loc[unidade].tolist() if unidade in df_pivot.index else [0]*len(anos_para_carregar)
        dados_formatados.append([nome_final, ''] + valores_ano)

    # Totais Consolidados
    dados_formatados.append(['Total', ''] + df_pivot.sum().tolist())

    df_final_graduacao_evolucao_csv = pd.DataFrame(dados_formatados)
    display(df_final_graduacao_evolucao_csv)
else:
    print("❌ Erro: Falha ao consolidar dados históricos.")
    df_final_graduacao_evolucao_csv = pd.DataFrame()

--- Processando: Graduação (Evolução Histórica) ---
⏳ Iniciando coleta de série histórica (2019-2024)...


,0,1,2,3,4,5,6,7
0,Evolução do número de Alunos Matriculados de G...,,,,,,,None
1,Unidades,Número - Anos,,,,,,None
2,,,2019,2020,2021,2022,2023,2024
3,EESC,,2706,2683,2713,2619,2585,2596
4,IAU,,289,288,287,289,288,285
5,ICMC,,1208,1245,1277,1295,1315,1306
6,IFSC,,430,448,454,475,480,470
7,IQSC,,280,276,301,307,306,306
8,Programas Conjuntos,,,,,,,
9,EESC e ICMC - Eng. Computação,,301,304,300,315,304,310


## **PÓS-GRADUAÇÃO**

# 9. Pós-Graduação: Alunos Matriculados (Ano Base)
Levantamento do total de alunos matriculados (Mestrado e Doutorado) no ano de referência, segmentado por unidade e incluindo o programa interunidades em Bioengenharia.

In [24]:
print("--- Processando: Pós-Graduação (Alunos Atuais) ---")
df_pos = carregar_tabela_anuario_usp(ANO_REFERENCIA, CONFIG_TABELAS['servidores'])

if df_pos is not None:
    col_num = df_pos.columns[-1]

    # Filtra alunos de Pós
    df_alunos_pos = df_pos[
        (df_pos['Humanos'] == 'Alunos cadastrados') &
        (df_pos['Docentes'].astype(str).str.contains('Pós-graduação|Pós-Graduação', case=False, regex=True)) &
        (df_pos['Recursos'] == 'Atividades de Ensino e Pesquisa')
    ].copy()

    dados_qtd = [['Alunos de Pós-Graduação', ''], ['Unidades', 'Número']]
    total_pos = 0

    # Unidades Regulares
    for un in UNIDADES_SAO_CARLOS:
        val = df_alunos_pos[df_alunos_pos['EACH'] == un][col_num]
        n = int(val.iloc[0]) if not val.empty else 0
        dados_qtd.append([un, n])
        total_pos += n

    # Bioengenharia
    dados_qtd.append(['Programas Conjuntos', ''])
    val_bio = df_alunos_pos[df_alunos_pos['EACH'] == 'Interunidades em Bioengenharia'][col_num]
    n_bio = int(val_bio.iloc[0]) if not val_bio.empty else 0
    dados_qtd.append(['Bioengenharia', n_bio])
    total_pos += n_bio

    dados_qtd.append(['Total', total_pos])

    df_pos_graduacao = pd.DataFrame(dados_qtd)
    display(df_pos_graduacao)

--- Processando: Pós-Graduação (Alunos Atuais) ---
⬇️ Download concluído: tabela_T1.02_2024.xls


,0,1
0,Alunos de Pós-Graduação,
1,Unidades,Número
2,EESC,1134
3,IAU,280
4,ICMC,883
5,IFSC,384
6,IQSC,341
7,Programas Conjuntos,
8,Bioengenharia,49
9,Total,3071


## **PÓS-GRADUAÇÃO - AVALIAÇÃO CAPES**

# 10. Pós-Graduação: Qualidade (Avaliação CAPES)
Extração do indicador de qualidade dos programas de pós-graduação, medido pelo percentual de cursos com conceito entre 4 e 7 na avaliação da CAPES.

In [25]:
print("--- Processando: Pós-Graduação (Avaliação CAPES) ---")
# Recarregamos ou reutilizamos a tabela base
if 'df_pos' not in locals():
    df_pos = carregar_tabela_anuario_usp(ANO_REFERENCIA, CONFIG_TABELAS['servidores'])

if df_pos is not None:
    col_num = df_pos.columns[-1]
    dados_capes = [['Alunos de Pós-Graduação', ''], ['Avaliação CAPES', ''], ['Unidades', 'Conceitos de 4 a 7 (%)']]

    def get_capes(nome_each):
        f = df_pos[
            (df_pos['EACH'] == nome_each) &
            (df_pos['Recursos'] == 'Atividades de Ensino e Pesquisa') &
            (df_pos['Humanos'] == 'Avaliação CAPES') &
            (df_pos['Docentes'] == 'Conceitos de 4 a 7 (%)')
        ][col_num]
        return f"{float(f.iloc[0]):.2f}" if not f.empty else "0.00"

    for un in UNIDADES_SAO_CARLOS:
        dados_capes.append([un, get_capes(un)])

    dados_capes.append(['Programas Conjuntos', ''])
    dados_capes.append(['Bioengenharia', get_capes('Interunidades em Bioengenharia')])

    df_capes_avaliacao = pd.DataFrame(dados_capes)
    display(df_capes_avaliacao)

--- Processando: Pós-Graduação (Avaliação CAPES) ---


,0,1
0,Alunos de Pós-Graduação,
1,Avaliação CAPES,
2,Unidades,Conceitos de 4 a 7 (%)
3,EESC,100.00
4,IAU,100.00
5,ICMC,100.00
6,IFSC,100.00
7,IQSC,100.00
8,Programas Conjuntos,
9,Bioengenharia,100.00


## **PÓS-GRADUAÇÃO - EVOLUÇÃO**

# 11. Pós-Graduação: Evolução Histórica
Extração complexa da série histórica de títulos outorgados (Mestrado e Doutorado) consolidando dados de 1934 até o presente.

In [26]:
print("--- Processando: Pós-Graduação (Evolução Histórica) ---")

# Função local especializada para T3.12
def obter_dados_pos_evolucao(ano_alvo):
    cod_tabela = CONFIG_TABELAS['pos_graduacao_evolucao']

    # Lógica de Roteamento: Histórico vs Ano Específico
    ano_download = ANO_REFERENCIA if ano_alvo == '1934 a 2020' else int(ano_alvo)

    df = carregar_tabela_anuario_usp(ano_download, cod_tabela, verbose=False)
    if df is None: return None

    mapa_nomes = {'ICMC e UFSCar': 'ICMC-UFSCar'}
    dados_ano = {}

    for unidade in UNIDADES_POS_GRAD:
        nome_tabela = mapa_nomes.get(unidade, unidade)

        # Filtro composto: Ano de Referência na linha + Unidade
        df_u = df[
            (df['Ano Referência'].astype(str) == str(ano_alvo)) &
            (df['Unidade'] == nome_tabela)
        ]

        # Extração segura com try/except para cada nível
        try: m = int(df_u[df_u['Nível de Pós-graduação'] == 'Mestrado']['Total'].iloc[0])
        except: m = 0
        try: d = int(df_u[df_u['Nível de Pós-graduação'] == 'Doutorado']['Total'].iloc[0])
        except: d = 0

        dados_ano[unidade] = {'Mestrado': m, 'Doutorado': d, 'Total': m + d}

    return dados_ano

# --- Execução ---
anos_evolucao = ['1934 a 2020', '2021', '2022', '2023', '2024']
cache_dados = {}

print("⏳ Coletando dados históricos T3.12...")
for ano in anos_evolucao:
    cache_dados[ano] = obter_dados_pos_evolucao(ano)

# --- Construção da Matriz Final ---
tabela = []
# Cabeçalhos multinível
tabela.append(['Evolução de Títulos de Pós-Graduação outorgados'] + [''] * (len(anos_evolucao)*3 + 1))
tabela.append(['Unidades', 'Número - Anos'] + [''] * (len(anos_evolucao)*3))

header_anos = ['', '']
for ano in anos_evolucao: header_anos.extend([ano, '', ''])
header_anos.extend(['Total', ''])
tabela.append(header_anos)

header_tipos = ['', '']
for _ in anos_evolucao: header_tipos.extend(['Total', 'Mest.', 'Dout.'])
header_tipos.extend(['Mest.', 'Dout.'])
tabela.append(header_tipos)

# Preenchimento
for unidade in UNIDADES_POS_GRAD:
    if unidade == 'ICMC e UFSCar':
        tabela.append(['Programas Conjuntos'] + [''] * (len(anos_evolucao)*3 + 2))

    linha = [unidade, '']
    soma_mest_geral, soma_dout_geral = 0, 0

    for ano in anos_evolucao:
        d = cache_dados.get(ano, {}).get(unidade, {'Total':0, 'Mestrado':0, 'Doutorado':0})
        linha.extend([d['Total'], d['Mestrado'], d['Doutorado']])
        soma_mest_geral += d['Mestrado']
        soma_dout_geral += d['Doutorado']

    linha.extend([soma_mest_geral, soma_dout_geral])
    tabela.append(linha)

df_final_pos_graduacao_evolucao_csv = pd.DataFrame(tabela)
display(df_final_pos_graduacao_evolucao_csv)

--- Processando: Pós-Graduação (Evolução Histórica) ---
⏳ Coletando dados históricos T3.12...


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,Evolução de Títulos de Pós-Graduação outorgados,,,,,,,,,,,,,,,,,None,None
1,Unidades,Número - Anos,,,,,,,,,,,,,,,,None,None
2,,,1934 a 2020,,,2021,,,2022,,,2023,,,2024,,,Total,
3,,,Total,Mest.,Dout.,Total,Mest.,Dout.,Total,Mest.,Dout.,Total,Mest.,Dout.,Total,Mest.,Dout.,Mest.,Dout.
4,EESC,,8285,5759,2526,236,148,88,226,129,97,206,105,101,199,112,87,6253,2899
5,IAU,,292,180,112,24,15,9,25,19,6,44,27,17,47,26,21,267,165
6,ICMC,,2733,1908,825,117,66,51,115,66,49,124,70,54,134,83,51,2193,1030
7,IFSC,,1337,695,642,58,30,28,68,44,24,71,49,22,72,39,33,857,749
8,IQSC,,1840,977,863,60,26,34,58,27,31,47,17,30,66,28,38,1075,996
9,Programas Conjuntos,,,,,,,,,,,,,,,,,,None


## **EXTENSÃO - ESPECIALIZAÇÃO**

# 12. Extensão: Cursos de Especialização
Dados referentes aos cursos Especialização, contabilizando o número de participantes.

In [27]:
print("--- Processando: Extensão (Especialização) ---")
df_esp = carregar_tabela_anuario_usp(ANO_REFERENCIA, CONFIG_TABELAS['extensao_especializacao'])

if df_esp is not None:
    col_esp = df_esp.columns[-1]
    dados_esp = [['Alunos de Extensão - Especialização', ''], ['Unidades', 'Número de participantes']]
    tot_esp = 0

    for un in UNIDADES_SAO_CARLOS:
        v = df_esp[(df_esp['Curso'] == 'Especialização') & (df_esp['Unidade'] == un)][col_esp]
        n = int(v.iloc[0]) if not v.empty else 0
        dados_esp.append([un, n])
        tot_esp += n

    dados_esp.append(['Total', tot_esp])

    df_final_extensao_csv = pd.DataFrame(dados_esp)
    display(df_final_extensao_csv)

--- Processando: Extensão (Especialização) ---
⬇️ Download concluído: tabela_T2.23_2024.xls


,0,1
0,Alunos de Extensão - Especialização,
1,Unidades,Número de participantes
2,EESC,60
3,IAU,0
4,ICMC,1158
5,IFSC,0
6,IQSC,0
7,Total,1218


## **EXTENSÃO - PRESENCIAL**

# 13. Extensão: Cursos Presenciais
Levantamento de cursos de difusão e atualização oferecidos na modalidade presencial, com número de cursos e participantes.

In [28]:
print("--- Processando: Extensão (Presencial) ---")
df_pres = carregar_tabela_anuario_usp(ANO_REFERENCIA, CONFIG_TABELAS['extensao_presencial'])

if df_pres is not None:
    dados_pres = [['Alunos de Extensão - Presencial', '', ''], ['Unidades', 'Cursos', 'Participantes']]
    tot_c, tot_p = 0, 0

    for un in UNIDADES_EXTENSAO:
        if un == 'Interunidades em Bioengenharia': dados_pres.append(['Programas Conjuntos', '', ''])

        vc = df_pres[(df_pres['Unidade'] == un) & (df_pres['Categoria'] == 'Cursos') & (df_pres['Ano'] == ANO_REFERENCIA)]['Total']
        vp = df_pres[(df_pres['Unidade'] == un) & (df_pres['Categoria'] == 'Participantes') & (df_pres['Ano'] == ANO_REFERENCIA)]['Total']

        nc = int(vc.iloc[0]) if not vc.empty else 0
        np = int(vp.iloc[0]) if not vp.empty else 0

        nome = 'Bioengenharia' if un == 'Interunidades em Bioengenharia' else un
        dados_pres.append([nome, nc, np])
        tot_c += nc
        tot_p += np

    dados_pres.append(['Total', tot_c, tot_p])

    df_final_extensao_presencial_csv = pd.DataFrame(dados_pres)
    display(df_final_extensao_presencial_csv)

--- Processando: Extensão (Presencial) ---
⬇️ Download concluído: tabela_T3.15_2024.xls


,0,1,2
0,Alunos de Extensão - Presencial,,
1,Unidades,Cursos,Participantes
2,EESC,14,786
3,IAU,6,122
4,ICMC,11,923
5,IFSC,0,0
6,IQSC,4,91
7,PUSP-SC,6,80
8,Programas Conjuntos,,
9,Bioengenharia,0,0


## **EXTENSÃO - À DISTÂNCIA**

# 14. Extensão: Educação a Distância (EAD)
Levantamento de cursos de extensão oferecidos na modalidade à distância.

In [29]:
print("--- Processando: Extensão (EAD) ---")
df_ead = carregar_tabela_anuario_usp(ANO_REFERENCIA, CONFIG_TABELAS['extensao_distancia'])

if df_ead is not None:
    dados_ead = [['Alunos de Extensão - EAD', '', ''], ['Unidades', 'Cursos', 'Participantes']]
    tot_c, tot_p = 0, 0

    for un in UNIDADES_EXTENSAO:
        if un == 'Interunidades em Bioengenharia': dados_ead.append(['Programas Conjuntos', '', ''])

        vc = df_ead[(df_ead['Unidade'] == un) & (df_ead['Categoria'] == 'Cursos') & (df_ead['Ano'] == ANO_REFERENCIA)]['Total']
        vp = df_ead[(df_ead['Unidade'] == un) & (df_ead['Categoria'] == 'Participantes') & (df_ead['Ano'] == ANO_REFERENCIA)]['Total']

        nc = int(vc.iloc[0]) if not vc.empty else 0
        np = int(vp.iloc[0]) if not vp.empty else 0

        nome = 'Bioengenharia' if un == 'Interunidades em Bioengenharia' else un
        dados_ead.append([nome, nc, np])
        tot_c += nc
        tot_p += np

    dados_ead.append(['Total', tot_c, tot_p])

    df_final_extensao_a_distancia_csv = pd.DataFrame(dados_ead)
    display(df_final_extensao_a_distancia_csv)

--- Processando: Extensão (EAD) ---
⬇️ Download concluído: tabela_T3.35_2024.xls


,0,1,2
0,Alunos de Extensão - EAD,,
1,Unidades,Cursos,Participantes
2,EESC,1,60
3,IAU,0,0
4,ICMC,24,29658
5,IFSC,0,0
6,IQSC,1,0
7,PUSP-SC,0,0
8,Programas Conjuntos,,
9,Bioengenharia,0,0


# 15. Carga: Exportação de Dados (CSV)
Pipeline final que salva todas as 12 tabelas geradas acima em arquivos CSV na pasta organizada.

In [30]:
print(f"--- Iniciando Pipeline de Exportação -> {DIRETORIO_SAIDA} ---")

# Mapeamento: Variável do Notebook -> Nome do arquivo final
export_map = {
    'df_area': 'area_territorial.csv',
    'df_final_servidores_csv': 'servidores.csv',
    'df_final_docentes_csv': 'docentes.csv',
    'df_final_professores_seniores_csv': 'professores_seniores.csv',
    'df_final_graduacao_csv': 'graduacao.csv',
    'df_final_graduacao_evolucao_csv': 'graduacao_evolucao.csv',
    'df_pos_graduacao': 'pos_graduacao.csv',
    'df_capes_avaliacao': 'pos_graduacao_avaliacao_capes.csv',
    'df_final_pos_graduacao_evolucao_csv': 'pos_graduacao_evolucao.csv',
    'df_final_extensao_csv': 'extensao_especializacao.csv',
    'df_final_extensao_presencial_csv': 'extensao_presencial.csv',
    'df_final_extensao_a_distancia_csv': 'extensao_ead.csv'
}

sucesso = 0
falha = 0

for var_name, filename in export_map.items():
    # Verifica se a variável existe no contexto global
    if var_name in globals():
        df = globals()[var_name]
        if isinstance(df, pd.DataFrame) and not df.empty:
            path = os.path.join(DIRETORIO_SAIDA, filename)
            df.to_csv(path, index=False, header=False)
            print(f"✅ [OK] {filename}")
            sucesso += 1
        else:
            print(f"⚠️ [VAZIO] {filename} - DataFrame sem dados.")
            falha += 1
    else:
        print(f"❌ [ERRO] {filename} - Variável '{var_name}' não foi gerada.")
        falha += 1

print(f"\nResumo: {sucesso} arquivos gerados, {falha} falhas.")

--- Iniciando Pipeline de Exportação -> dados_2024 ---
✅ [OK] area_territorial.csv
✅ [OK] servidores.csv
✅ [OK] docentes.csv
✅ [OK] professores_seniores.csv
✅ [OK] graduacao.csv
✅ [OK] graduacao_evolucao.csv
✅ [OK] pos_graduacao.csv
✅ [OK] pos_graduacao_avaliacao_capes.csv
✅ [OK] pos_graduacao_evolucao.csv
✅ [OK] extensao_especializacao.csv
✅ [OK] extensao_presencial.csv
✅ [OK] extensao_ead.csv

Resumo: 12 arquivos gerados, 0 falhas.
